In [12]:
from mlx_lm import load, generate
from mlx_lm_lora.trainer.datasets import CacheDataset, ChatDataset
from datasets import load_dataset
from mlx_lm_lora.trainer.sft_trainer import SFTTrainingArgs, train_sft, evaluate_sft
import mlx.optimizers as optim
from mlx_lm.utils import save_config
from mlx_lm.tuner.callbacks import TrainingCallback

from pathlib import Path
import path_file

In [4]:
# At this point, the model has to be 'Instruct'
model_name = "mlx-community/Qwen2.5-Coder-1.5B-Instruct-Q6"
model, tokenizer = load(model_name)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
# test_out = generate(model, tokenizer, prompt="def f(x : int) -> str:\n    ")

In [6]:
dataset_path = path_file.dataset_path

In [7]:
# !python3 -m mlx_lm.lora --train --model mlx-community/Qwen2.5-Coder-1.5B-Instruct-Q6 --data train2 --batch-size 1


In [8]:
import mlx.core as mx
import numpy as np

def beam_search(model, tokenizer, prompt, beam_width=5, max_new_tokens=50):
    # encode prompt
    input_ids = mx.array([tokenizer.encode(prompt)])

    # initialize beams
    beams = [(input_ids, 0.0)]  # (tokens, logprob)

    for _ in range(max_new_tokens):
        new_beams = []

        for tokens, logprob in beams:
            logits = model(tokens).logits[:, -1, :]  # last token logits
            probs = mx.softmax(logits, axis=-1)
            probs = np.array(probs[0])  # to numpy for top-k
            topk_ids = probs.argsort()[-beam_width:][::-1]  # best tokens

            for tok in topk_ids:
                tok_prob = np.log(probs[tok] + 1e-9)
                new_tokens = mx.concatenate([tokens, mx.array([[tok]])], axis=1)
                new_beams.append((new_tokens, logprob + tok_prob))

        # keep only best beams
        new_beams.sort(key=lambda x: x[1], reverse=True)
        beams = new_beams[:beam_width]

    # return final N sequences
    results = [
        tokenizer.decode(np.array(tokens[0].tolist()))
        for (tokens, score) in beams
    ]
    return results


In [15]:
model, tokenizer = load(
    "mlx-community/Qwen2.5-Coder-1.5B-Instruct-Q6",
    adapter_path='/Users/artem.semidetnov/Documents/Predictor/adapters'
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [16]:
sequences = beam_search(
    model=model,
    tokenizer=tokenizer,
    prompt="Explain quantum entanglement:",
    beam_width=3,
    max_new_tokens=60
)

for i, seq in enumerate(sequences):
    print(f"\nSequence {i+1}:")
    print(seq)

AttributeError: 'array' object has no attribute 'logits'